# 11장 - 종양 탐지를 위한 분류 모델 훈련

## 1절 - 기본 모델과 훈련 루프

In [ ]:
# 2부의 나머지 부분에서 더 큰 프로젝트에 필요한 도구가 될 분류 모델과 훈련 루프 만들 예정
# 이를 위해 Ct 클래스와 LunaDataset 클래스를 DataLoader 인스턴스에 삽입
# 이후 인스턴스를 데이터와 함께 훈련 루프와 검증 루프를 거쳐 분류 모델에 입력

In [ ]:
# 다음으로 훈련 루프 실행 결과를 사용하기 위해 여러 제약 있는 데이터를 어떻게 고품질의 결과로 만드는지 설명
# 이후 장에서는 데이터의 제약 사항 완화할 방안 확인

In [ ]:
# 각 후보는 결절인지 아닌지로 분류하며, 모델에 전달할 각 샘플에 대해 종류별로 레이블 할당해야 함
# 각 샘플에 있는 후보에 '결절' 혹은 '비결절' 레이블 매겨짐

In [ ]:
# 시작부터 끝까지 연결된 엔드투엔드 파이프라인을 초기에 만들어 놓으면 좋은 이정표가 됨
# 결과를 분석적으로 평가할 수 있을 만큼 잘 작동한다면, 매번 변경으로 결과가 향상된다는 확신을 가질 수 있음
# 적어도 효과 없는 변경이나 실험은 차선으로 미룰 수 있음

In [ ]:
# 구현의 기본 구조
# 모델 초기화 및 데이터 로딩
# 임의로 선택한 에포크 수로 루프 반복
#   - LunaDataset이 반환한 훈련 데이터의 배치 루프 돌기
#   - 백그라운드에서 데이터로도 워커 프로세스를 적합한 배치 읽어들이기
#   - 배치를 분류 모델에 전달해 결과 얻기
#   - 추정 결과를 실측 데이터와 비교하여 손실 계산
#   - 임시 데이터 구조에 모델의 성능 메트릭 기록
#   - 오차 역전파로 모델 가중치 조정
#   - 검증 데이터 배치로 루프 반복
#   - 백그라운드 워커 프로세스에서 검증 데이터 배치 읽어들이기
#   - 배치 분류하고 손실 계산
#   - 모델이 검증 데이터에 대해 얼마나 잘 동작했는지 기록
#   - 매 에포크마다 진행 상황과 성능 정보 출력

In [ ]:
# 전보다 복잡해진 만큼 구조적으로 프로그램 작성해야 함
# 프로젝트는 잘 만들어진 여러 함수를 훈련 애플리케이션에서 사용하며, 데이터셋 등을 위한 코드는 자체적인 파이썬 모듈로 분리

In [ ]:
# 자신의 프로젝트 수행할 때는 복잡도에 어울리는 구조나 설계 수준 택해야 함
# 충분한 구조화 없으면 실험을 깔끔하게 수행하거나 문제 해결하기 어려우며, 자신이 무엇을 하는지 설명하기도 쉽지 않음
# 과한 구조화는 쓸모없는 인프라 구조 작성에 시간을 낭비하게 만들며, 그 틀에 맞추기 위한 여러 작업을 하느라 진행이 더딜 것임

In [ ]:
# 훈련 진척에 대한 다양한 메트릭 수집할 것임
# 조흥 메트리 로깅 없이 훈련에 준 변화가 어떤 영향을 미치는지 파악하기 어려움
# 올바른 메트릭 수집하는 것의 중요성 이야기 할 것
# 메트릭을 수집하기 위한 인프라 구조 깔고 전체와 개별 클래스 단위로 손실값과 잘 분류된 샘플의 백분율을 수집하고 표시할 것임

## 2절 - 애플리케이션의 메인 진입점

In [ ]:
# 2부는 완전한 명령행 애플리케이션임
# 명령행 인자를 파싱하고 완전한 --help 옵션 제공해 여러 환경에서 쉽게 실행 가능

In [ ]:
# 애플리케이션은 클래스로 구현해 필요할 때 인스턴스로 만들어 실행함
# 운영체제 수준에서 별도 프로세스로 띄우지 않아도 애플리케이션 구동 가능

In [ ]:
# 함수 호출이나 OS 레벨의 프로세스로 훈련 구동 가능하면 함수 호출을 래핑해 주피터 노트북에 넣을 수 있으므로 코드를 쉽게 네이티브 CLI나 브라우저에서 호출할 수 있는 장점 존재

In [ ]:
# code/p2_run_everything.ipynb
def run(app, *argv):
    argv = list(argv)
    argv.insert(0, '--num-workers=4')  # <1>
    log.info("Running: {}({!r}).main()".format(app, argv))

    app_cls = importstr(*app.rsplit('.', 1))  # <2>
    app_cls(argv).main()

    log.info("Finished: {}.{!r}).main()".format(app, argv))

In [ ]:
# 파일 끝부분에 애플리케이션 객체를 인스턴스로 만들고 main 메소드를 호출하는 표준 형태의 if main 코드 확인 가능

In [ ]:
# training.py:386
if __name__ == '__main__':
    LunaTrainingApp().main()

In [ ]:
# 처음의 애플리케이션 클래스를 보면 위에서 호출한 두 함수 __init__과 main 볼 수 있음
# 명령행 인자를 받으므로 애플리케이션 __init__ 함수에서 표준 argparse 라이브러리를 사용
# 초기화 함수에 커스텀 인자를 전달하 수 있어야만 함
# main 메소드는 애플리케이션의 핵심 로직을 위한 진입점임

In [ ]:
# training.py:31
class LunaTrainingApp:
    def __init__(self, sys_argv=None):
        if sys_argv is None:
            sys_argv = sys.argv[1:]

        parser = argparse.ArgumentParser()
        parser.add_argument('--num-workers',
            help='Number of worker processes for background data loading',
            default=8,
            type=int,
        )

        # 63행
        self.cli_args = parser.parse_args(sys_argv)
        self.time_str = datetime.datetime.now().strftime('%Y-%m-%d_%H.%M.%S')

        # 137행
        def main(self):
            log.info("Starting {}, {}".format(type(self).__name__, self.cli_args))

In [ ]:
# 매우 일반적인 구조이므로 향후 다른 프로젝트에서 재사용 가능
# __init__ 안에서의 인자 파싱은 애플리케이션 호출에 영향 받지 않고 설정 가능

## 3절 - 사전 훈련 설정과 초기화

In [ ]:
# 에포크 내의 각 배치 순회하기 전 초기화가 필요
# 모델과 옵티마이저의 초기화와 Dataset과 DataLoader 인스턴스 초기화로 나눌 수 있음
# LunaDataset은 랜덤으로 선택한 샘플셋을 정의해 훈련 에포크 채울 것이고 DataLoader 인스턴스는 데이터셋으로부터 데이터를 읽는 작업 수행하여 애플리케이션에 제공

#### .1 모델과 옵티마이저 초기화

In [ ]:
# 이번 절에서는 LunaModel 세부를 블랙박스로 생각
# 시작 부분 확인

In [ ]:
# training.py:31
class LunaTrainingApp:
    def __init__(self, sys_argv=None):

        # 70행
        self.use_cuda = torch.cuda.is_available()
        self.device = torch.device("cuda" if self.use_cuda else "cpu")

        self.model = self.initModel()
        self.optimizer = self.initOptimizer()

    def initModel(self):
        model = LunaModel()
        if self.use_cuda:
            log.info("Using CUDA; {} devices.".format(torch.cuda.device_count()))
            if torch.cuda.device_count() > 1: # 복수의 GPU 탐지
                model = nn.DataParallel(model) # 모델 래핑
            model = model.to(self.device) # GPU에 모델 파라미터 전달
        return model

    def initOptimizer(self):
        return SGD(self.model.parameters(), lr=0.001, momentum=0.99)

In [ ]:
# 복수의 GPU 사용시 nn.DataParallel 클래스 사용해 모든 GPU에 작업을 분산하여 처리 후 파라미터를 모아 조정 작업을 재동기화하는 방식으로 진행됨

In [ ]:
# self.use_cuda값이 True면 self.model.to(device) 호출로 모델 파라미터를 GPU로 옮기고 컨볼루션과 숫자 연산에 GPU 사용하도록 설정
# 옵티마이저 구성 전에 설정하지 않는다면 옵티마이저는 CPU 상의 파라미터 객체를 봄

In [ ]:
# 모멘텀과 함께 SGD 사용
# SGD는 대부분의 문제에 적용가능하며, lr=0.001과 momentum=0.99도 안전한 선택에 해당하는 값임
# 잘 안되면 lr을 0.1이나 0.0001로 변경해볼만 함

#### 2. 데이터 로더의 관리와 데이터 공급

In [ ]:
# LunaDataset 클래스는 원본 데이터와 파이토치 빌딩 블럭을 위한 구조화된 텐서 사이의 가교 역할
# LunaDataet.__getitem__에 있던 ct_t.unsqueeze(0)는 데이터의 네 번째 차원인 채널을 제공함
# 단일한 밀도를 가지므로 채널 차원 크기는 1

In [ ]:
# 단일 샘플에 대한 훈련이나 검증 작업 수행은 대부분의 플랫폼에서 이미 여러 샘플에 대한 병렬 계산을 지원하기 때문에 컴퓨터 자원을 비효율적으로 사용함
# 개선을 위해 여러 셈플을 배치 튜플로 묶어 한 번에 처리될 수 있게 해야 함
# 5차원(N)을 동일한 배치 내에서 샘플을 구별하는 용도로 사용

In [ ]:
# DataLoader 클래스가 배치를 구현하였고 CT 스캔을 텐서로 바꾸기 위해 LunaDataset 클래스를 만들었으니 데이터셋을 데이터 로더에 연결하면 됨

In [ ]:
# training.py:89
    def initTrainDl(self):
        train_ds = LunaDataset( # 커스텀 데이터셋
            val_stride=10,
            isValSet_bool=False,
        )

        batch_size = self.cli_args.batch_size
        if self.use_cuda:
            batch_size *= torch.cuda.device_count()

        train_dl = DataLoader( # 바로 사용하면 되는 클래스
            train_ds,
            batch_size=batch_size, # 자동으로 배치로 나뉨
            num_workers=self.cli_args.num_workers,
            pin_memory=self.use_cuda, # 고정된 메모리 영역이 GPU로 빠르게 전송됨
        )

        return train_dl

    # 137행
    def main(self):
        log.info("Starting {}, {}".format(type(self).__name__, self.cli_args))

        train_dl = self.initTrainDl()
        val_dl = self.initValDl() # 검증 데이터 로더는 훈련 데이터 로더와 유사

In [ ]:
# 데이터 로더는 개별 샘플을 배치로 만들고, 별도의 프로세스와 공유 메모리를 사용한 병렬 로딩도 제공
# 데이터 로더 인스턴스 만들 때 num_workers=만 지정하면 나머지 알아서 처리되고 각 워커 프로세스는 배치 생성함

In [ ]:
# 코드에서 for batch_tup in self.train_dl: 처럼 루프 돌 때 매번 Ct를 로드하고 샘플을 가져와 배치할 때까지 기다릴 필요 없음
# 이미 로딩 끝난 batch_tup을 바로 사용하며 워커 프로세스는 다음 순회 시 사용하기 위한 또 다른 배치를 로드하기 위해 자동으로 해제될 것
# 파이토치 데이터 로딩 기능 사용 시 GPU 계산과 데이터 로딩을 동시에 진행하므로 대부분의 경우 프로젝트 빠르게 실행 가능

## 4절 - 첫 번째 경로 신경망 설계

#### .1 핵심 컨볼루션

In [ ]:
# 분류 모델에서는 테일, 백본(=바디), 헤드로 구성된 구조가 흔함
# 테일(tail)은 입력을 신경망에 넣기 전 처리 과정을 담당하는 제일 첫 부분의 일부 계층
# 백본이 원하는 형태로 입력을 만들어야 하므로 신경망의 나머지 부분과는 구조나 구성이 다른 경우가 많음
# 우리는 단순 배치 정규화 계층 사용

In [ ]:
# 백본은 여러 계층을 가지는데 일반적으로 연속된 블럭에 배치됨
# 각 블럭은 동일한 세트의 계층을 가지며 블럭을 거칠 때마다 필요한 입력 크기나 필터 수가 달라짐
# 우리는 두 개의 3*3 컨볼루션과 하나의 활성함수, 맥스 풀링 연산이 이어진 블럭 사용

In [ ]:
# 블럭 코드 model.py:67
class LunaBlock(nn.Module):
    def __init__(self, in_channels, conv_channels):
        super().__init__()

        self.conv1 = nn.Conv3d(
            in_channels, conv_channels, kernel_size=3, padding=1, bias=True,
        )
        self.relu1 = nn.ReLU(inplace=True)
        self.conv2 = nn.Conv3d(
            conv_channels, conv_channels, kernel_size=3, padding=1, bias=True,
        )
        self.relu2 = nn.ReLU(inplace=True)

        self.maxpool = nn.MaxPool3d(2, 2)

    def forward(self, input_batch):
        block_out = self.conv1(input_batch)
        block_out = self.relu1(block_out)
        block_out = self.conv2(block_out)
        block_out = self.relu2(block_out)

        return self.maxpool(block_out)

In [ ]:
# 헤드는 백본의 출력을 받아 원하는 출력 형태로 변환
# 컨볼루션 신경망에서 이 작업은 중간 출력물을 평탄화 하고 완전 연결 계층에 전달하는 역할을 하기도 함
# 우리의 분류는 두 가지뿐이므로 하나의 평탄화 계층만 사용

In [ ]:
# 이 블럭에서는 3*3*3 컨볼루션 사용 -> 27개의 복셀이 입력되고 1개를 출력함
# 컨볼루션층이 쌓여 있어 마지막 출력 복셀은 커널의 크기보다 입력의 영향을 받음
# 두 데이터의 최종 출력은 실질적으로 5*5*5 수용 필드를 가져 조금 큰 하나의 컨볼루션층처럼 동작
# 결과적으로 두 층의 3*3*3 계층은 5*5*5 컨볼루션보다 적은 파리미터를 가지고 계산이 더 빠름

In [ ]:
# 두 층의 컨볼루션의 출력은 2*2*2 맥스 풀링으로 들어감
# 6*6*6 수용 필드에서 7/8에 해당하는 데이터 버리고 가장 큰 값 가지는 한 개의 5*5*5필드 생성하는 것임
# 버려지는 입력 복셀도 맥스 풀링을 통해 출력 픽셀 하나에 대해 결국은 중첩되는 입력 필드가 연결되므로 어떤 식으로든 최종 출력에 영향 줌

In [ ]:
# 각 컨볼루션층을 통해 수용 필드가 줄어들지만 패딩을 사용하고 있어 입력과 출력 이미지 크기는 동일함

In [ ]:
# 위 블록은 여러 번 반복하며 모델의 백본을 구성함

#### .2 전체 모델

In [ ]:
# model.py:13
class LunaModel(nn.Module):
    def __init__(self, in_channels=1, conv_channels=8):
        super().__init__()

        # 테일
        self.tail_batchnorm = nn.BatchNorm3d(1)

        # 백본
        self.block1 = LunaBlock(in_channels, conv_channels)
        self.block2 = LunaBlock(conv_channels, conv_channels * 2)
        self.block3 = LunaBlock(conv_channels * 2, conv_channels * 4)
        self.block4 = LunaBlock(conv_channels * 4, conv_channels * 8)

        # 헤드
        self.head_linear = nn.Linear(1152, 2)
        self.head_softmax = nn.Softmax(dim=1)

NameError: name 'nn' is not defined

In [ ]:
# model.py:13
class LunaModel(nn.Module):
    def __init__(self, in_channels=1, conv_channels=8):
        super().__init__()

        self.tail_batchnorm = nn.BatchNorm3d(1)

        self.block1 = LunaBlock(in_channels, conv_channels)
        self.block2 = LunaBlock(conv_channels, conv_channels * 2)
        self.block3 = LunaBlock(conv_channels * 2, conv_channels * 4)
        self.block4 = LunaBlock(conv_channels * 4, conv_channels * 8)

        self.head_linear = nn.Linear(1152, 2)
        self.head_softmax = nn.Softmax(dim=1)

In [ ]:
# 테일은 nn.BatchNorm3d를 사용해 입력을 정규화 함
# nn.BatchNorm3d는 입력값을 이동시키고 비율을 조정해 평균이 0이고 표준 정규분포 1을 따르게 함
# 특이한 하운스필드 단위는 신경망 뒤에서는 보이지 않음
# 입력 단위를 알고 있고 관련 조직이 가져야 하는 값이 무엇인지 파악하고 있어 수정된 정규화를 손쉽게 구현 가능

In [ ]:
# 백본에는 4개의 반복 블럭이 있고 nn.Module 서브클래스를 각각 구분하여 사용
# 각 블럭은 2*2*2 맥스 풀링 연산으로 끝나므로 이미지는 16배 줄어든 형태가 나옴
# 32*48*48 이미지가 입력이었으므로 2*3*3이 됨

In [ ]:
# 테일에는 완전 연결 계층 뒤로 nn.Softmax가 있음
# 소프트맥스는 단일 레이블 분류에서 유용한 함수
# 출력을 0~1 사이값으로 묶어서 입력의 절댓값이 커져도 영향받지 않으며 입력 내 상대적인 값에만 영향 받으므로 정답이 얼마나 확실한지 표현하기 좋음
# 모델에서는 파이토치 버전인 nn.Softmax를 사용해 배치나 텐서에 호환되고 자동미분도 빠르게 실행 가능

In [ ]:
# self.block4의 출력을 완전 연결 계층에 넣지 못하는 문제 발생
# 출력은 샘플마다 2*3*3이미지에 64개 채널이지만 완전 연결 계층의 입력은 1차원 벡터

In [ ]:
# forward 메소드 확인
# model.py:50
def forward(self, input_batch):
        bn_output = self.tail_batchnorm(input_batch)

        block_out = self.block1(bn_output)
        block_out = self.block2(block_out)
        block_out = self.block3(block_out)
        block_out = self.block4(block_out)

        conv_flat = block_out.view(
            block_out.size(0), # 배치 크기
            -1,
        )
        linear_output = self.head_linear(conv_flat)

        return linear_output, self.head_softmax(linear_output)

In [ ]:
# view 함수를 이용해 평탄화 수행 후 데이터 전달
# 연산은 동작에 영향 미치는 파라미터 저장을 요구하지 않으므로 forward 함수 내에서 연산 실행
# 컨볼루션 사용해 분류나 회귀 혹은 이미지가 아닌 출력 결과를 만드는 대부분의 모델은 신경망의 헤드에 유사한 요소 가짐

In [ ]:
# forward 메소드는 출력을 위해 logit과 소프트맥스로 확률 만듦
# logit은 소프트맥스 계층에 들어가기 전의 입력임
# 훈련 중에는 nn.CrossEntropyLoss 계산을 위해 로지트 값을 사용하며 실제 분류할 때는 확률값 사용
# 소프트맥스처럼 상태값이 따로 없고 두 값의 차이가 단순한 경우에는 훈련 때와 실제 제품으로 사용할 때가 다른 경우 많음

In [ ]:
# 모델이 좋은 성능 낼 수 있도록 동작하려면 모델의 가중치, 편향값, 여러 파라미터가 특정 속성을 드러내야 함
# 여러 정규화 기술로 계층의 출력을 잘 동작하게 만들기도 하지만 신경망 가중치 초기화가 가장 단순한 방법
# 파이토치는 초기화를 보완하는 수단을 제공하지 않으므로 직접 해야 함

In [ ]:
# _init_weights 함수를 기본으로 사용 가능
# model.py:30
def _init_weights(self):
    for m in self.modules():
        if type(m) in {
            nn.Linear,
            nn.Conv3d,
            nn.Conv2d,
            nn.ConvTranspose2d,
            nn.ConvTranspose3d,
        }:
            nn.init.kaiming_normal_(
                m.weight.data, a=0, mode='fan_out', nonlinearity='relu',
            )
            if m.bias is not None:
                fan_in, fan_out = \
                    nn.init._calculate_fan_in_and_fan_out(m.weight.data)
                bound = 1 / math.sqrt(fan_out)
                nn.init.normal_(m.bias, -bound, bound)

## 5절 - 모델 훈련과 검증

In [ ]:
# 훈련 루프 코드
# training.py:137
def main(self):

    # 143행
    for epoch_ndx in range(1, self.cli_args.epochs + 1):
        trnMetrics_t = self.doTraining(epoch_ndx, train_dl)
        self.logMetrics(epoch_ndx, 'trn', trnMetrics_t)

    # 165행
def doTraining(self, epoch_ndx, train_dl):
        self.model.train()
        trnMetrics_g = torch.zeros( # 빈 메트릭 배열 초기화
            METRICS_SIZE,
            len(train_dl.dataset),
            device=self.device,
        )

        batch_iter = enumerateWithEstimate(
            train_dl,
            "E{} Training".format(epoch_ndx),
            start_ndx=train_dl.num_workers,
        )
        for batch_ndx, batch_tup in batch_iter:
            self.optimizer.zero_grad() # 남은 가중치 텐서 해제

            loss_var = self.computeBatchLoss(
                batch_ndx,
                batch_tup,
                train_dl.batch_size,
                trnMetrics_g
            )

            loss_var.backward() # 모델 가중치 조정
            self.optimizer.step()

        self.totalTrainingSamples_count += len(train_dl.dataset)

        return trnMetrics_g.to('cpu')

In [ ]:
# trnMetrics_g 텐서가 훈련 중 자세한 클래스 단위 메트릭 수집함
# 규모가 큰 프로젝트에서는 위 값으로 종요한 통찰 얻는 경우가 많기 때문에 필요
# 완료 시간 예측을 제공하기 위해 enumerateWithEstimate 사용
# 실제 손실 계산이 computeBatchLoss 메소드에서 이뤄지게 함으로써 코드를 재사용함

In [ ]:
# trnMetics_g 텐서의 목적은 각 샘프레 대해 computeBatchLoss 함수부터 logMetrics 함수까지 어떻게 동작했는지에 대한 정보 전달하는 것

#### .1 computeBatchLoss 함수

In [ ]:
# computeBatchLoss 함수는 훈련 루프와 검증 루프 양쪽에서 호출됨
# 샘플 배치에 대해 손실을 계산하는 함수
# 부가적으로 각 샘플에 대해 모델이 만들어내는 출력에 대한 정보도 계산해서 기록하여, 각 클래스별로 계산이 얼마나 정확한지 백분율로 계산 가능하며 분류가 잘 되지 않는 클래스를 찾아 집중 개선 가능

In [ ]:
# computeBatchLoss 함수의 핵심 기능은 배치 단위로 모델에 입력 넣고 손실을 계산하는 것이므로 CrossEntropyLoss 사용
# 배치 튜플의 패킹 풀고 텐서를 GPU로 옮긴 후 모델 호출하는 과정임

In [ ]:
# training.py:225
def computeBatchLoss(self, batch_ndx, batch_tup, batch_size, metrics_g):
    input_t, label_t, _series_list, _center_list = batch_tup

    input_g = input_t.to(self.device, non_blocking=True)
    label_g = label_t.to(self.device, non_blocking=True)

    logits_g, probability_g = self.model(input_g)

    loss_func = nn.CrossEntropyLoss(reduction='none')
    loss_g = loss_func( # reduction='none'으로 샘플별 손실값 얻음
        logits_g,
        label_g[:,1], # 원핫 인코딩 클래스의 인덱스
    )

    # 248행
    return loss_g.mean() # 샘플별 손실값을 단일값으로 합침

In [ ]:
# 손실값이 들어있는 텐서를 샘플마다 얻어 개별 손실값 추적 가능하고 원하는 방식으로 병합 가능하게 함
# 위 코드에서는 각 샘플에 대한 손실 평균을 반환해 배치 단위의 손실과 동일한 값을 넘김
# 샘플별 통계값을 가질 필요없으면 배치 단위의 손실값 평균 계산 사용해도 문제 없음

In [ ]:
# 역전파 단계 가기 전에 추후 분석을 위해 샘플별 통계값 기록
# training.py:26
METRICS_LABEL_NDX=0 # 인덱스를 위한 변수 선언은 모듈 범위에서 유효함
METRICS_PRED_NDX=1
METRICS_LOSS_NDX=2
METRICS_SIZE = 3

# 225행
def computeBatchLoss(self, batch_ndx, batch_tup, batch_size, metrics_g):
    # 238행
    start_ndx = batch_ndx * batch_size
    end_ndx = start_ndx + label_t.size(0)
    # 기울기에 의존적인 메트릭 없으므로 detach 사용
    metrics_g[METRICS_LABEL_NDX, start_ndx:end_ndx] = \
        label_g[:,1].detach()
    metrics_g[METRICS_PRED_NDX, start_ndx:end_ndx] = \
        probability_g[:,1].detach()
    metrics_g[METRICS_LOSS_NDX, start_ndx:end_ndx] = \
        loss_g.detach()

    return loss_g.mean()

In [ ]:
# 모든 훈련과 검증 샘플에 대해 레이블, 예측 결과, 손실값을 기록해 모델 동작 조사에 사용하기 위한 풍부한 세부 정보 확보함
# 클래스 단위의 통계를 모으기 위함과 이상하게 분류된 샘플을 찾아서 왜 그런지를 쉽게 확인하는데 사용

#### .2 훈련 때와 유사한 검증 루프

In [ ]:
# 검증 루프는 읽기 전용이라는 점과 손실값을 사용하지 않으며 가중치도 조정하지 않는다는 차이점을 가짐
# 모델에 대해 아무것도 바꾸면 안되며, with torch.no_grad() 콘텍스트 매니저가 명시적으로 기울기 계산이 불필요하다고 하므로 훈련 때보다 빠름

In [ ]:
# training.py:137
def main(self):
    for epoch_ndx in range(1, self.cli_args.epochs + 1):

        # 157행
        valMetrics_t = self.doValidation(epoch_ndx, val_dl)
        self.logMetrics(epoch_ndx, 'val', valMetrics_t)

# 203행
def doValidation(self, epoch_ndx, val_dl):
    with torch.no_grad():
        self.model.eval()
        valMetrics_g = torch.zeros(
            METRICS_SIZE,
            len(val_dl.dataset),
            device=self.device,
        )

        batch_iter = enumerateWithEstimate(
            val_dl,
            "E{} Validation ".format(epoch_ndx),
            start_ndx=val_dl.num_workers,
        )
        for batch_ndx, batch_tup in batch_iter:
            self.computeBatchLoss(
                batch_ndx, batch_tup, val_dl.batch_size, valMetrics_g)

    return valMetrics_g.to('cpu')

In [ ]:
# 신경망 가중치 조정하지 않고 computeBatchLoss가 주는 손실값 사용하지 않고 옵티마이저 참조도 하지 않아 루프에는 computeBatchLoss 호출만 남음
# computBatchLoss 호출로 넘어오는 배치의 전체 손실값을 사용하지 않지만 호출의 부수효과로 valMetrics_g에 메트릭 계속 모음

## 6절 - 성능 메트릭 출력

In [ ]:
# 에포크의 마지막 작업은 성능 메트릭 로깅하는 것
# 각 에포크마다 진행 상태 로깅을 위해 결과를 trnMetrics_g와 valMerics_g에 수집하였음

#### .1 logMetrics 함수

In [ ]:
# logMetrics 함수의 시그니처
# training.py:251
def logMetrics(
        self,
        epoch_ndx,
        mode_str,
        metrics_t,
        classificationThreshold=0.5,
):

In [ ]:
# epoch_ndx는 결과를 로깅할 때 표시하기 위한 용도
# mode_str 인자는 메트릭이 훈련용인지 검증용인지 표시
# 훈련이면 trnMetrics_t, 검증이면 valMetrics_t을 metrics_t 파라미터로 전달
# 두 입력모두 computeBatchLoss를 통해 만들어진 부동소수점 데이터 텐서이며 doTraining이나 doBalidation으로부터 반환 직전에 CPU 영역으로 전송됨
# 둘 다 세 개의 행과 샘플 수 만큼의 열 가짐
# 세 개 행은 다음의 상수와 동일

In [ ]:
# training.py:26
METRICS_LABEL_NDX=0
METRICS_PRED_NDX=1
METRICS_LOSS_NDX=2
METRICS_SIZE = 3

In [ ]:
# 결절 샘플 또는 비결절 샘플에 대해서만 메트릭 제한하는 마스크 제작하고 클래스별 총 샘플 수와 잘 분류된 샘플의 수 세기
# training.py:264
negLabel_mask = metrics_t[METRICS_LABEL_NDX] <= classificationThreshold
negPred_mask = metrics_t[METRICS_PRED_NDX] <= classificationThreshold

posLabel_mask = ~negLabel_mask
posPred_mask = ~negPred_mask

In [ ]:
# 결절의 상태 레이블은 True 혹은 False이므로 metrics_t[METRICS_LABEL_NDX]에 저장된 값은 {0.0, 1.0} 범위에 속함
# 기본값 0.5로 설정한 classificationThreshold와 비교해 이진값 배열을 얻게 되면 True는 비결절(음성)인 경우의 레이블에 해당
# negPred_mask 만들기 위한 비교도 수행함
# METRICS_PRED_NDX 값은 모델이 만든 양의 예측값이며 0.0 ~ 1.0인 부동소수점 값
# 비교 방식은 동일하지만 실젯값이 0.5에 가까워질 수 있음

In [ ]:
# 레이블별 통계를 계산하기 위해 마스크를 사용하고 metrics_dict에 결과 저장
# training.py:270
neg_count = int(negLabel_mask.sum())
pos_count = int(posLabel_mask.sum())

neg_correct = int((negLabel_mask & negPred_mask).sum())
pos_correct = int((posLabel_mask & posPred_mask).sum())

metrics_dict = {}
metrics_dict['loss/all'] = \
    metrics_t[METRICS_LOSS_NDX].mean()
metrics_dict['loss/neg'] = \
    metrics_t[METRICS_LOSS_NDX, negLabel_mask].mean()
metrics_dict['loss/pos'] = \
    metrics_t[METRICS_LOSS_NDX, posLabel_mask].mean()

metrics_dict['correct/all'] = (pos_correct + neg_correct) \
    / np.float32(metrics_t.shape[1]) * 100
metrics_dict['correct/neg'] = neg_correct / np.float32(neg_count) * 100
metrics_dict['correct/pos'] = pos_correct / np.float32(pos_count) * 100

In [ ]:
# 에포크 전체에 대한 평균 손실값 계산
# 손실값은 훈련 중 최소화되고 있는 단일값이므로 추적할 필요 있음
# negLabel_mask 사용해 음성으로 레이블 된 샘플에 대해서만 손실값의 평균 구하도록 제한하고 posLabel_mask를 사용해 양성에 대한 손실값도 계산
# 이처럼 클래스별 계산을 해두면 특정 클래스가 다른 클래스보다 분류하기 어려운 경우를 추적하여 개선하기 좋음

In [ ]:
# 마지막으로 전체 샘플 중 얼마나 정확하게 맞추었는지와 각 레이블별 정확도를 계산
# 손실값과 마찬가지로, 이 값들은 개선을 위해 어디에 노력을 기울여야 할지 안내하는 역할
# 계산 끝나면, 세 번의 log.info 호출을 통해 결과 로깅

In [ ]:
# training.py:289
log.info(
    ("E{} {:8} {loss/all:.4f} loss, "
         + "{correct/all:-5.1f}% correct, "
    ).format(
        epoch_ndx,
        mode_str,
        **metrics_dict,
    )
)
log.info(
    ("E{} {:8} {loss/neg:.4f} loss, "
         + "{correct/neg:-5.1f}% correct ({neg_correct:} of {neg_count:})"
    ).format(
        epoch_ndx,
        mode_str + '_neg',
        neg_correct=neg_correct,
        neg_count=neg_count,
        **metrics_dict,
    )
)
log.info(
    ("E{} {:8} {loss/pos:.4f} loss, "
        + "{correct/pos:-5.1f}% correct ({pos_correct:} of {pos_count:})"
    ).format(
        epoch_ndx,
        mode_str + '_pos',
        pos_correct=pos_correct,
        pos_count=pos_count,
        **metrics_dict,
    )
)

In [ ]:
# 첫 번째 로그는 모든 샘플에 대해 계산하므로 /all로 태그되었고, 음성은 /neg, 양성은 /pos로 태그됨

## 7절 - 훈련 스크립트 실행

In [ ]:
# 초기화 수행 후 모델 훈련시켜 잘 훈련되고 있는지 통계값 출력할 것
# 스크립트는 메인 코드가 들어 있는 디렉토리에서 실행
# 디렉토리 아래에 하위 디렉토리가 있어야 함
# 사용하는 python 환경은 requirements.txt에 명시한 모든 라이브러리 설치된 상태여야 함
$ python -m p2ch11.training

In [ ]:
# 훈련 애플리케이션 실행시켜주는 주피터 노트북을 우리가 제공한다는 사실도 기억
run('p2ch11.prepcache.LunaPrepCacheApp')
run('p2ch11.training.LunaTrainingApp', '--epochs=1')

In [ ]:
# 훈련이 시작되면 예상대로 가용한 컴퓨팅 자원을 사용하는지 확인할 필요 있음
# 병목 현상이 데이터 로딩 때문인지 연산 때문인지를 구분하려면 스크립트가 실행되어 훈련을 시작할 때까지 잠시 기다린 후 top과 nvidia-smi로 점검
# 만일 8개의 파이썬 워커 프로세스가 80% 이상의 CPU 사용한다면 캐시가 필요한 상황
# 만일 nvidia-smi에서 GPU-Util이 80% 이상이면 GPU는 풀 가둥인 것임

In [ ]:
# 여러 에포크를 최대한 빨리 끝내기 위해 가능한 한 많은 컴퓨팅 파워를 사용해야 함
# 단일 1080 Ti로 에포크 하나 끝내는 데 15분이 안 걸림
# 모델이 상대적으로 단순해 CPU가 병목이 될 만큼 전처리를 위해 많은 CPU 자원을 사용하지 않음
# 좀 더 심층 모델을 다룬다면 각 배치를 처리하는데 많은 시간이 걸릴 것이고, 다음 배치 입력이 준비되지 않아 GPU가 할일 없는 상황이 발생하지 않도록 CPU가 처리해야 할 일의 양도 늘어남

#### .1 훈련에 필요한 데이터

In [ ]:
# 샘플 수가 적다면 전체 데이터 사용하고 있는지 점검할 필요 있음
# data-unversioned/part2/luna 디렉토리의 기본 디렉토리 구조 확인
# 각 시리즈 UID에 대해 .mhd파일과 .raw파일이 하나씩 있는지 확인
# 파일 수 정확한지 비교
# 다 정확한데 잘 동작하지 않는다면 매닝 라이브북(Manning LiveBook) 사이트에 질문

#### .2 막간을 활용해 enumerateWithEstimate 함수 알아보기

In [ ]:
# util.py:143
def enumerateWithEstimate(
        iter,
        desc_str,
        start_ndx=0,
        print_ndx=4,
        backoff=None,
        iter_len=None,
):
    for (current_ndx, item) in enumerate(iter):
        yield (current_ndx, item)

In [ ]:
# enumerateWithEstimate 함수 사용법과 결과
for i, _ in enumerateWithEstimate(list(range(234)), "sleeping"):

SyntaxError: incomplete input (<ipython-input-25-84af89ae6784>, line 2)

In [ ]:
# enumerateWithEstimate의 동작은 enumerate와 거의 동일(리턴 값이 다름)

In [ ]:
# 딥러닝 프로젝트는 결국 시간 싸움
# 언제 작업이 끝날지를 안다면 시간을 효율적으로 사용 가느아며 뭔가 제대로 동작하지 않을 때(또는 접근 방법이 잘못됐을 때) 완료까지 예상보다 훨씬 오래 걸리는 것에서 힌트를 얻어 문제 파악 가능함

## 8절 - 모델 평가: 정확도 99.7%라면 잘 끝난 것일까?

In [ ]:
# 훈련 결과를 확인하니 문제가 발생함
# 결과를 쉽게 이해할 수 있도록 도구 업그레이드 -> 얻은 메트릭으로 그래프 그리기

## 9절 - 텐서보드로 훈련 관련 메트릭를 그려보기

In [ ]:
# 훈련 루프에서 만들어진 훈련 메트릭을 그래프로 만들기 위해 텐서보드 사용
# 전체적인 추세 확인 가능
# 텐서보드로 메트릭 시각화하면 해당 값이 메트릭의 추세를 따르는지, 이상값인지 구분하기 훨씬 쉬워짐

In [ ]:
# 텐서보드는 훌륭하며 파이토치 API로 데이터를 연결해서 빠르고 쉽게 표시할 수 있음

#### .1 텐서보드 실행

In [ ]:
# 기본값으로 훈련 스크립트는 메트릭 데이터를 runs/ 하위 디렉토리에 써내려 감

In [ ]:
# tensorboard 프로그램은 tensorflow 파이썬 패키지를 설치하면 얻을 수 있음
# 텐서보드가 사용할 데이터는 별도 폴더로 분리해 놓으면 좋음

In [ ]:
# Scalars 탭에서 왼쪽 메뉴는 데스플레이 옵션과 현재 실행 중인 리스트가 보임
# 스무딩 옵션은 데이터에 노이즈 많은 경우 유용하며 적용 정도를 선택해 전체 추세를 부드럽게 조절 가능
# 훈련 스크립트 실행한 만큼 실행 아이템이 보이고 선택할 수 있음

In [ ]:
# 특정 실행 아이템을 완전히 지우면 텐서보드는 해당 아이템을 디스크에서 삭제함
# 잘못 실행됐거나 오류가 나서 수렴하지 않는 등 필요 없는 경우에 지우면 됨
# 실행 횟수는 상당히 빠르게 늘어나므로 자주 확인하고 삭제하거나 이름을 알맞게 바꾸고 중요 결과는 특정 디렉토리에 옳겨서 실수로 지우지 않게 함

#### .2 메트릭 로깅 함수가 텐서보드를 지원하도록 만들기

In [ ]:
# 텐서보드가 읽을 수 있는 포맷으로 데이터 저장하려면 torch.utils.tensorboard 모듈 사용해 빠르고 쉽게 메트릭 저장
# 텐서보드는 넘파이 배열과 텐서를 지원하므로 텐서 사용

In [ ]:
# 먼저 torch.utils.tensorboard에서 임포트 하는 SummaryWriter 객체 생성
# 전달할 파라미터는 runs/p2ch11/2020-01-01_12.55.27-trn-dlwpt 같은 것을 초기화할 때 필요한 log_dir 뿐
# dlwpt가 정보 제공하도록 변경하기 위해 훈련 스크립트에 주석 인자 더할 수도 있음

In [ ]:
# 쓰기 객체를 훈련 실행 결과와 검증 실행 결과에 사용하도록 두 개 만들기
# 쓰기 객체는 모든 에포크에서 재사용
# SummaryWriter 클래스 초기화하면 log_dir 디렉토리도 만들어짐
# 디텍토리는 텐서보드에 나타나며, 데이터가 만들어지기도 전에 훈련 스크립트 수행이 실패하면 UI가 실행 아이템으로 어지럽혀짐
# 너무 많은 빈 UI 실행 아이템이 생기지 않게 데이터가 쓸 만한 단계까지 실행되었을 때 SummaryWriter 객체를 초기화하면 됨
# initTensorBoardWriter() 함수는 logMetrics()에서 호출함

In [ ]:
# training.py:127
def initTensorboardWriters(self):
    if self.trn_writer is None:
        log_dir = os.path.join('runs', self.cli_args.tb_prefix, self.time_str)

        self.trn_writer = SummaryWriter(
            log_dir=log_dir + '-trn_cls-' + self.cli_args.comment)
        self.val_writer = SummaryWriter(
            log_dir=log_dir + '-val_cls-' + self.cli_args.comment)

In [ ]:
# 훈련 루프는 최초에 랜덤하게 초기화된 상태로 실행되므로 첫 에포크의 실행값은 이상하므로 첫 배치에 대한 메트릭 저장하면 결국 값을 왜곡시킴
# 텐서보드는 스무딩을 통해 조금이라도 도움되도록 이런 노이즈 줄이는 기능 제공함

In [ ]:
# 스칼라값 기록은 직관적임
# 이미 만든 metrics_dict의 키/값을 writer.add_scalar 메소드에 전달

In [ ]:
# torch.utils.tensorboard.SummaryWriter 클래스의 add_scalar 메소드 시그니처
def add_scalar(self, tag, scalar_value, global_step=None, walltime=None):
    # ...

SyntaxError: incomplete input (<ipython-input-38-9607465055b7>, line 3)

In [ ]:
# tag 파라미터는 텐서보드의 어떤 그래프에 값이 들어가는지를 알려주며 scalar_value 파라미터는 데이터 포인트의 Y축 값을 의미하며 global_step 파라미터는 X축 값으로 동작
# doTraining 함수 안에서 totalTrainingSamples_count 변수를 업데이트했던 것을 상기
# 이 변수를 global_step 파라미터로 텐서보드에 넘겨 그래프의 X축으로 사용

In [ ]:
# training.py:323
for key, value in metrics_dict.items():
    writer.add_scalar(key, value, self.totalTrainingSamples_count)

In [ ]:
# loss/all처럼 키에 /가 들어가면 텐서보드는 / 전까지의 문자열을 잘라 그룹으로 만듦

## 10절 - 모델이 결절 탐지를 학습하지 못하는 이유

In [ ]:
# 에포크 증가에 따라 손실값이 일정한 추세를 보이고 결과는 재현 가능하므로 학습하고 있는 것은 명확
# 모델에게 기대하는 학습과 실제 모델 학습에는 괴리가 있음

In [ ]:
# 현재 모델은 모든 질문에 거짓이라고 답하는 단순한 방식으로 동작

In [ ]:
# 훈련셋이나 검증셋에 대해 손실값이 줄어들고 있음
# 문제에 대한 추적할 수 있는 값을 가지고 있으므로 희망이 있고 다음 장에서의 과제임

## 11절 - 결론

In [ ]:
# 이제 이전 장에서 만들었던 데이터를 읽어 훈련 루프를 실행하는 모델 존재
# 메트릭은 콘솔에 로깅 결과로 나오며 시작적인 그래프 보여줌
# 결과는 쓸 만하지 않아 진행 상황을 추적하기 위한 메트릭을 더 개선하여 모델이 합리적인 결과 내려면 어디를 바꿔야 할지 알려주도록 만들 예정

## 12절 - 연습 문제

In [ ]:
# 1. DataLoader 인스턴스 내에 래핑한 LunaDataset 인스턴스를 순회하는 프로그램을 만들어라. 순회할 때 걸리는 시간도 알 수 있게 만들어서 10장의 연습 문제와 시간을 비교해보자. 스크립트를 실행할 때 캐시 상태를 인지하기 바란다.
#   a. num_worker=를 0, 1, 2로 바꿀 때 어떤 차이가 발생하는가?
#   b. 메모리가 모자라지 않는 선에서 최대로 끌어올릴 수 있는 batch_size=와 num_workers=는 얼마인가

In [ ]:
# 2. noduleInfo_list의 정렬 순서를 반대로 해보자. 이렇게 바꾸면 훈련의 첫 에포크 후에 동작 방식에서 어떤 차이가 발생하는가?

In [ ]:
# 3. logMetrics를 바꿔서 텐서보드가 사용하는 실행 아이템 이름과 키를 변경해보자.
#   a. writer.add_scalar로 전달되는 키 값에 처음으로 나타나는 슬래시 문자 위치를 바꿔보자.
#   b. 동일한 쓰기 객체를 사용해서 훈련과 검증에 대해 돌려본 후 키 이름에 trn이나 val 문자열을 덧붙여보자.
#   c. 로그 디렉토리와 키 값을 원하는 대로 바꿔보자.